In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Import Quarterback performance dataset

Collect data

In [2]:
career_stats_passing = pd.read_csv("Career_Stats_Passing.csv")

In [3]:
career_stats_passing

,Player Id,Name,Position,Year,Team,Games Played,Passes Attempted,Passes Completed,Completion Percentage,Pass Attempts Per Game,...,TD Passes,Percentage of TDs per Attempts,Ints,Int Rate,Longest Pass,Passes Longer than 20 Yards,Passes Longer than 40 Yards,Sacks,Sacked Yards Lost,Passer Rating
0,tomfarris/2513861,"Farris, Tom",NaN,1948,Chicago Rockets,0,--,--,--,0.0,...,--,--,--,--,--,--,--,--,--,0.0
1,tomfarris/2513861,"Farris, Tom",NaN,1947,Chicago Bears,9,2,0,0.0,0.2,...,0,0.0,0,0.0,--,0,0,0,0,39.6
2,tomfarris/2513861,"Farris, Tom",NaN,1946,Chicago Bears,11,21,8,38.1,1.9,...,1,4.8,2,9.5,--,0,0,0,0,31.5
3,billdemory/2512778,"Demory, Bill",NaN,1974,New York Jets,1,--,--,--,0.0,...,--,--,--,--,--,--,--,--,--,0.0
4,billdemory/2512778,"Demory, Bill",NaN,1973,New York Jets,6,39,12,30.8,6.5,...,2,5.1,8,20.5,--,0,0,8,73,22.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8520,billanderson/2508534,"Anderson, Bill",NaN,1962,Washington Redskins,12,--,--,--,0.0,...,--,--,--,--,--,--,--,--,--,0.0
8521,billanderson/2508534,"Anderson, Bill",NaN,1961,Washington Redskins,14,--,--,--,0.0,...,--,--,--,--,--,--,--,--,--,0.0
8522,billanderson/2508534,"Anderson, Bill",NaN,1960,Washington Redskins,12,--,--,--,0.0,...,--,--,--,--,--,--,--,--,--,0.0
8523,billanderson/2508534,"Anderson, Bill",NaN,1959,Washington Redskins,11,--,--,--,0.0,...,--,--,--,--,--,--,--,--,--,0.0


In [4]:
career_stats_passing.head()

,Player Id,Name,Position,Year,Team,Games Played,Passes Attempted,Passes Completed,Completion Percentage,Pass Attempts Per Game,...,TD Passes,Percentage of TDs per Attempts,Ints,Int Rate,Longest Pass,Passes Longer than 20 Yards,Passes Longer than 40 Yards,Sacks,Sacked Yards Lost,Passer Rating
0,tomfarris/2513861,"Farris, Tom",NaN,1948,Chicago Rockets,0,--,--,--,0.0,...,--,--,--,--,--,--,--,--,--,0.0
1,tomfarris/2513861,"Farris, Tom",NaN,1947,Chicago Bears,9,2,0,0.0,0.2,...,0,0.0,0,0.0,--,0,0,0,0,39.6
2,tomfarris/2513861,"Farris, Tom",NaN,1946,Chicago Bears,11,21,8,38.1,1.9,...,1,4.8,2,9.5,--,0,0,0,0,31.5
3,billdemory/2512778,"Demory, Bill",NaN,1974,New York Jets,1,--,--,--,0.0,...,--,--,--,--,--,--,--,--,--,0.0
4,billdemory/2512778,"Demory, Bill",NaN,1973,New York Jets,6,39,12,30.8,6.5,...,2,5.1,8,20.5,--,0,0,8,73,22.2


Career_stats_passing dataset contains information about the quarterbacks performance

In [5]:
qb_performance = career_stats_passing.copy()

Convert string columns to numerical data types

In [6]:
numerical_columns = ['Passes Completed', 'Passes Attempted', 'Passing Yards', 'TD Passes', 'Ints']
for col in numerical_columns:
    qb_performance[col] = pd.to_numeric(qb_performance[col], errors='coerce')

# Feature Engineering

In [7]:
qb_performance['Completion Percentage'] = qb_performance['Passes Completed'] / qb_performance['Passes Attempted']
qb_performance['Yards per Attempt'] = qb_performance['Passing Yards'] / qb_performance['Passes Attempted']
qb_performance['Touchdown per Attempt'] = qb_performance['TD Passes'] / qb_performance['Passes Attempted']
qb_performance['Interception per Attempt'] = qb_performance['Ints'] / qb_performance['Passes Attempted']
qb_performance['Passing Efficiency'] = qb_performance['Completion Percentage'] * qb_performance['Yards per Attempt']
qb_performance['TD to INT Ratio'] = qb_performance['TD Passes'] / qb_performance['Ints']
qb_performance['TD to Completion Ratio'] = qb_performance['TD Passes'] / qb_performance['Passes Completed']
qb_performance['INT to Completion Ratio'] = qb_performance['Ints'] / qb_performance['Passes Completed']

Replace infinity values with NaN and Fill any NaN values with 0

In [8]:
qb_performance.replace([np.inf, -np.inf], np.nan, inplace=True)

In [9]:
qb_performance.fillna(0, inplace=True)

Group by player and year to calculate the average performance over time

In [10]:
qb_performance_grouped = qb_performance.groupby(['Player Id', 'Year']).mean().reset_index()
qb_performance_grouped

,Player Id,Year,Games Played,Passes Attempted,Passes Completed,Completion Percentage,Pass Attempts Per Game,Passing Yards,TD Passes,Ints,Passer Rating,Yards per Attempt,Touchdown per Attempt,Interception per Attempt,Passing Efficiency,TD to INT Ratio,TD to Completion Ratio,INT to Completion Ratio
0,a.j.feeley/2504566,2001,1.0,14.0,10.0,0.714286,14.0,143.0,2.0,1.0,114.0,10.214286,0.142857,0.071429,7.295918,2.000000,0.200000,0.100000
1,a.j.feeley/2504566,2002,6.0,154.0,86.0,0.558442,25.7,0.0,6.0,5.0,75.4,0.000000,0.038961,0.032468,0.000000,1.200000,0.069767,0.058140
2,a.j.feeley/2504566,2003,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,a.j.feeley/2504566,2004,11.0,356.0,191.0,0.536517,32.4,0.0,11.0,15.0,61.7,0.000000,0.030899,0.042135,0.000000,0.733333,0.057592,0.078534
4,a.j.feeley/2504566,2005,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8291,yalelary/2519113,1960,12.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8292,yalelary/2519113,1961,14.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8293,yalelary/2519113,1962,14.0,1.0,0.0,0.000000,0.1,0.0,0.0,0.0,39.6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8294,yalelary/2519113,1963,10.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Select features for the model

In [11]:
feature_columns = ['Completion Percentage', 'Yards per Attempt', 'Touchdown per Attempt', 'Interception per Attempt',
                   'Passing Efficiency', 'TD to INT Ratio', 'TD to Completion Ratio', 'INT to Completion Ratio']

# Random Forest Regressor

In [12]:
X = qb_performance[['Player Id'] + feature_columns]
y = qb_performance['Passer Rating']

In [13]:
X

,Player Id,Completion Percentage,Yards per Attempt,Touchdown per Attempt,Interception per Attempt,Passing Efficiency,TD to INT Ratio,TD to Completion Ratio,INT to Completion Ratio
0,tomfarris/2513861,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
1,tomfarris/2513861,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
2,tomfarris/2513861,0.380952,5.142857,0.047619,0.095238,1.959184,0.50,0.125000,0.250000
3,billdemory/2512778,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
4,billdemory/2512778,0.307692,4.076923,0.051282,0.205128,1.254438,0.25,0.166667,0.666667
...,...,...,...,...,...,...,...,...,...
8520,billanderson/2508534,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
8521,billanderson/2508534,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
8522,billanderson/2508534,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
8523,billanderson/2508534,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000


In [14]:
y

0        0.0
1       39.6
2       31.5
3        0.0
4       22.2
        ... 
8520     0.0
8521     0.0
8522     0.0
8523     0.0
8524     0.0
Name: Passer Rating, Length: 8525, dtype: float64

### Split the dataset into training and testing sets

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
# Store the 'Player Id' column separately
player_ids_train = X_train['Player Id']
player_ids_test = X_test['Player Id']

In [22]:
# Drop the 'Player Id' column from X_train and X_test
X_train = X_train.drop('Player Id', axis=1)
X_test = X_test.drop('Player Id', axis=1)

In [29]:
print(X_train.dtypes)

Completion Percentage       float64
Yards per Attempt           float64
Touchdown per Attempt       float64
Interception per Attempt    float64
Passing Efficiency          float64
TD to INT Ratio             float64
TD to Completion Ratio      float64
INT to Completion Ratio     float64
dtype: object


Train the Random Forest Regressor

In [23]:
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

Make predictions on the test set

In [24]:
y_pred = rf_regressor.predict(X_test)
y_pred

array([118.8       ,  81.118     ,   4.85965321, ...,   4.85965321,
       103.314     ,  57.556     ])

Evaluate the model's performance

In [25]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

In [26]:
print(f'Mean Squared Error: {mse:.2f}')
print(f'Root Mean Squared Error: {rmse:.2f}')
print(f'R2 Score: {r2:.2f}')

Mean Squared Error: 90.96
Root Mean Squared Error: 9.54
R2 Score: 0.95


### Display the first 10 predictions and actual passer ratings

In [27]:
player_id_to_name = qb_performance[['Player Id', 'Name']].set_index('Player Id').to_dict()['Name']
results_df = pd.DataFrame({"Player Id": player_ids_test.values,
                           "Name": [player_id_to_name[player_id] for player_id in player_ids_test.values],
                           "Actual Passer Rating": y_test.values.ravel(),
                           "Predicted Passer Rating": y_pred.ravel()})

In [28]:
results_df

,Player Id,Name,Actual Passer Rating,Predicted Passer Rating
0,joewashington/2528291,"Washington, Joe",118.8,118.800000
1,richgannon/2500754,"Gannon, Rich",79.8,81.118000
2,donmccauley/2520537,"McCauley, Don",0.0,4.859653
3,yalelary/2519113,"Lary, Yale",0.0,4.859653
4,demaryiusthomas/497328,"Thomas, Demaryius",0.0,4.859653
...,...,...,...,...
1700,vinceevans/2500575,"Evans, Vince",77.7,73.335000
1701,henrymarshall/2520201,"Marshall, Henry",0.0,4.859653
1702,bradmaynard/2501931,"Maynard, Brad",0.0,4.859653
1703,aaronrodgers/2506363,"Rodgers, Aaron",104.9,103.314000
